In [1]:
!pip install langchain langchain-community langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00


In [30]:
import os
import json
from datetime import datetime, timedelta
from typing import List, Dict, Any, Tuple
from dataclasses import dataclass
import requests
from langchain.llms.base import LLM
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate
from langchain import hub

In [45]:
@dataclass
class TouristSite:
    name: str
    lat: float
    lon: float
    outdoor: bool
    hours_needed: float
    start_time: float
    end_time: float
    day_text: str
    venue_open: int
    venue_closed: int
    day_raw: List[int]  # 24 hourly crowd levels
    date: str
    temperatures: List[float]  # 24 hourly temperatures

class DeepSeekLLM(LLM):
    """Custom LangChain LLM wrapper for DeepSeek API"""

    api_key: str
    model: str = "deepseek-chat"
    base_url: str = "https://api.deepseek.com/v1/chat/completions"

    def __init__(self, api_key: str, model: str = "deepseek-chat", **kwargs):
        super().__init__(api_key=api_key, model=model, **kwargs)

    @property
    def _llm_type(self) -> str:
        return "deepseek"

    def _call(self, prompt: str, stop=None, **kwargs) -> str:
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }

        data = {
            "model": self.model,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.1,
            "max_tokens": 2000
        }

        try:
            response = requests.post(self.base_url, headers=headers, json=data)
            response.raise_for_status()
            result = response.json()
            return result["choices"][0]["message"]["content"]
        except Exception as e:
            return f"Error calling DeepSeek API: {str(e)}"

class ItineraryOptimizer:
    def __init__(self, deepseek_api_key: str):
        self.llm = DeepSeekLLM(deepseek_api_key)
        self.current_datetime = None
        self.sites = []

    def parse_datetime(self, datetime_str: str) -> datetime:
        """Parse datetime string in various formats"""
        formats = [
            "%Y-%m-%d %H:%M:%S",
            "%Y-%m-%d %H:%M",
            "%Y-%m-%d",
            "%d/%m/%Y %H:%M",
            "%d-%m-%Y %H:%M"
        ]

        for fmt in formats:
            try:
                return datetime.strptime(datetime_str, fmt)
            except ValueError:
                continue
        raise ValueError(f"Unable to parse datetime: {datetime_str}")

    def load_schedule_data(self, schedule_data: List[Dict[str, Any]]) -> List[TouristSite]:
        """Convert schedule data to TouristSite objects"""
        sites = []
        for data in schedule_data:
            site = TouristSite(**data)
            sites.append(site)
        return sites

    def get_temperature_at_hour(self, site: TouristSite, hour: int) -> float:
        """Get temperature at specific hour for a site"""
        hour_index = int(hour) % 24
        return site.temperatures[hour_index]

    def get_crowd_level_at_hour(self, site: TouristSite, hour: int) -> int:
        """Get crowd level at specific hour for a site"""
        hour_index = int(hour) % 24
        return site.day_raw[hour_index]

    def is_venue_open(self, site: TouristSite, start_hour: float, end_hour: float) -> bool:
        """Check if venue is open during the proposed time slot"""
        venue_open = site.venue_open
        venue_closed = site.venue_closed

        # Handle venues that close after midnight
        if venue_closed < venue_open:  # e.g., open 9, close 1 (next day)
            return (start_hour >= venue_open or end_hour <= venue_closed) and \
                   (start_hour < 24 and end_hour <= 24 + venue_closed)
        else:
            return start_hour >= venue_open and end_hour <= venue_closed

    def calculate_penalty_score(self, site: TouristSite, start_hour: float, end_hour: float) -> float:
        """Calculate penalty score for scheduling a site at given time"""
        penalty = 0.0
        duration = end_hour - start_hour

        # Check each hour in the time slot
        for hour in range(int(start_hour), int(end_hour) + 1):
            hour_weight = min(1.0, (hour + 1 - start_hour) / duration)

            # Temperature penalty for outdoor sites
            if site.outdoor:
                temp = self.get_temperature_at_hour(site, hour)
                if temp > 33:
                    penalty += (temp - 33) * 10 * hour_weight  # Heavy penalty for high temps

            # Crowd penalty (always applies)
            crowd = self.get_crowd_level_at_hour(site, hour)
            penalty += crowd * 0.1 * hour_weight  # Moderate penalty for crowds

        return penalty

    def find_optimal_schedule(self, sites: List[TouristSite], current_time_hour: float) -> List[Dict[str, Any]]:
        """Find optimal schedule using a greedy approach with penalty minimization"""

        # Filter sites that haven't started yet
        upcoming_sites = [site for site in sites if site.start_time >= current_time_hour]

        if not upcoming_sites:
            return []

        # Generate all possible time slots for the day
        time_slots = []
        for hour in range(int(current_time_hour), 24):
            for minute in [0, 30]:  # Allow 30-minute intervals
                time_slots.append(hour + minute/60)

        # Find best schedule using optimization
        best_schedule = []
        remaining_sites = upcoming_sites.copy()
        current_time = current_time_hour

        while remaining_sites:
            best_site = None
            best_start_time = None
            best_penalty = float('inf')

            for site in remaining_sites:
                # Try different start times for this site
                for start_time in time_slots:
                    if start_time < current_time:
                        continue

                    end_time = start_time + site.hours_needed

                    # Check if venue is open
                    if not self.is_venue_open(site, start_time, end_time):
                        continue

                    # Check if we can fit all remaining sites after this one
                    remaining_duration = sum(s.hours_needed for s in remaining_sites)
                    available_time = 24 - end_time
                    if available_time < remaining_duration - site.hours_needed:
                        continue

                    # Calculate penalty for this scheduling
                    penalty = self.calculate_penalty_score(site, start_time, end_time)

                    if penalty < best_penalty:
                        best_penalty = penalty
                        best_site = site
                        best_start_time = start_time

            if best_site is None:
                # Fallback: schedule remaining sites in original order
                for site in remaining_sites:
                    end_time = current_time + site.hours_needed
                    if self.is_venue_open(site, current_time, end_time):
                        best_schedule.append({
                            'name': site.name,
                            'original_start_time': site.start_time,
                            'original_end_time': site.end_time,
                            'new_start_time': current_time,
                            'new_end_time': end_time,
                            'penalty_score': self.calculate_penalty_score(site, current_time, end_time),
                            'temperature_range': [
                                self.get_temperature_at_hour(site, int(current_time)),
                                self.get_temperature_at_hour(site, int(end_time))
                            ],
                            'crowd_range': [
                                self.get_crowd_level_at_hour(site, int(current_time)),
                                self.get_crowd_level_at_hour(site, int(end_time))
                            ]
                        })
                        current_time = end_time
                break

            # Schedule the best site
            end_time = best_start_time + best_site.hours_needed
            best_schedule.append({
                'name': best_site.name,
                'original_start_time': best_site.start_time,
                'original_end_time': best_site.end_time,
                'new_start_time': best_start_time,
                'new_end_time': end_time,
                'penalty_score': best_penalty,
                'temperature_range': [
                    self.get_temperature_at_hour(best_site, int(best_start_time)),
                    self.get_temperature_at_hour(best_site, int(end_time))
                ],
                'crowd_range': [
                    self.get_crowd_level_at_hour(best_site, int(best_start_time)),
                    self.get_crowd_level_at_hour(best_site, int(end_time))
                ],
                'outdoor': best_site.outdoor
            })

            remaining_sites.remove(best_site)
            current_time = end_time

        return best_schedule

    def create_tools(self):
        """Create tools for the LangChain agent"""

        def optimize_itinerary_tool(query: str) -> str:
            """Tool to optimize tourist itinerary based on current time"""
            try:
                # Extract current time from query if provided
                if "current time" in query.lower():
                    # Simple extraction - in real implementation, use NLP
                    parts = query.split()
                    for i, part in enumerate(parts):
                        if "time" in part.lower() and i < len(parts) - 1:
                            try:
                                time_str = parts[i + 1]
                                current_hour = float(time_str)
                                break
                            except:
                                current_hour = 10.0  # Default
                else:
                    current_hour = 10.0

                # Optimize schedule
                optimized_schedule = self.find_optimal_schedule(self.sites, current_hour)

                # Format results
                result = "OPTIMIZED ITINERARY:\n\n"
                total_penalty = 0

                for item in optimized_schedule:
                    start_hour = int(item['new_start_time'])
                    start_min = int((item['new_start_time'] - start_hour) * 60)
                    end_hour = int(item['new_end_time'])
                    end_min = int((item['new_end_time'] - end_hour) * 60)

                    result += f"🏛️ {item['name']}\n"
                    result += f"   Original: {item['original_start_time']:.1f} - {item['original_end_time']:.1f}\n"
                    result += f"   Optimized: {start_hour:02d}:{start_min:02d} - {end_hour:02d}:{end_min:02d}\n"
                    result += f"   Temperature: {item['temperature_range'][0]:.1f}°C - {item['temperature_range'][1]:.1f}°C\n"
                    result += f"   Crowd Level: {item['crowd_range'][0]} - {item['crowd_range'][1]}\n"
                    result += f"   Penalty Score: {item['penalty_score']:.2f}\n"
                    if item.get('outdoor', False):
                        result += f"   🌤️ OUTDOOR ACTIVITY\n"
                    result += "\n"

                    total_penalty += item['penalty_score']

                result += f"Total Penalty Score: {total_penalty:.2f}\n"
                result += f"(Lower scores indicate better optimization)\n"

                return result

            except Exception as e:
                return f"Error optimizing itinerary: {str(e)}"

        def analyze_conditions_tool(query: str) -> str:
            """Tool to analyze weather and crowd conditions"""
            try:
                result = "CURRENT CONDITIONS ANALYSIS:\n\n"

                for site in self.sites:
                    result += f"📍 {site.name}\n"
                    result += f"   Location: ({site.lat:.4f}, {site.lon:.4f})\n"
                    result += f"   Type: {'Outdoor' if site.outdoor else 'Indoor'}\n"
                    result += f"   Operating Hours: {site.venue_open}:00 - {site.venue_closed}:00\n"
                    result += f"   Duration Needed: {site.hours_needed} hours\n"

                    # Find peak temperature and crowd times
                    max_temp = max(site.temperatures)
                    max_temp_hour = site.temperatures.index(max_temp)
                    max_crowd = max(site.day_raw)
                    max_crowd_hour = site.day_raw.index(max_crowd)

                    result += f"   Peak Temperature: {max_temp:.1f}°C at {max_temp_hour}:00\n"
                    result += f"   Peak Crowd: {max_crowd} at {max_crowd_hour}:00\n"

                    # Find best times (low temp for outdoor, low crowd for all)
                    if site.outdoor:
                        # For outdoor sites, find hours with temp <= 33°C
                        good_hours = [i for i, temp in enumerate(site.temperatures)
                                    if temp <= 33 and site.venue_open <= i <= site.venue_closed]
                    else:
                        # For indoor sites, just consider venue hours
                        good_hours = list(range(site.venue_open, min(site.venue_closed + 1, 24)))

                    if good_hours:
                        # Sort by combined score (temperature + crowd)
                        scored_hours = []
                        for hour in good_hours:
                            temp_score = site.temperatures[hour] if site.outdoor else 0
                            crowd_score = site.day_raw[hour]
                            total_score = temp_score + crowd_score * 0.1
                            scored_hours.append((hour, total_score))

                        scored_hours.sort(key=lambda x: x[1])
                        best_hours = [h[0] for h in scored_hours[:3]]
                        result += f"   Best Times: {', '.join([f'{h}:00' for h in best_hours])}\n"

                    result += "\n"

                return result

            except Exception as e:
                return f"Error analyzing conditions: {str(e)}"

        return [
            Tool(
                name="optimize_itinerary",
                description="Optimize tourist itinerary to avoid high temperatures and crowds",
                func=optimize_itinerary_tool
            ),
            Tool(
                name="analyze_conditions",
                description="Analyze weather and crowd conditions for tourist sites",
                func=analyze_conditions_tool
            )
        ]

class TouristItineraryAgent:
    def __init__(self, deepseek_api_key: str):
        self.optimizer = ItineraryOptimizer(deepseek_api_key)
        self.agent_executor = None

    def setup_agent(self, schedule_data: List[Dict[str, Any]]):
        """Setup the LangChain agent with tools and data"""
        # Load schedule data
        self.optimizer.sites = self.optimizer.load_schedule_data(schedule_data)

        # Create tools
        tools = self.optimizer.create_tools()

        # Create agent prompt
        prompt = PromptTemplate(
            input_variables=["tools", "tool_names", "input", "agent_scratchpad"],
            template="""You are a tourist itinerary optimization agent. Your goal is to reschedule tourist activities to:
1. Avoid outdoor sites during hours when temperature > 33°C
2. Minimize exposure to crowded times at all sites
3. Ensure all sites can be visited within their operating hours
4. Maintain the required duration for each site

Available tools: {tool_names}
Tools: {tools}

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Question: {input}
Thought: {agent_scratchpad}"""
        )

        # Create agent
        agent = create_react_agent(self.optimizer.llm, tools, prompt)
        self.agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

    def optimize_schedule(self, current_datetime: str) -> str:
        """Main method to optimize the schedule"""
        if not self.agent_executor:
            return "Agent not setup. Call setup_agent() first."

        try:
            # Parse current datetime
            dt = self.optimizer.parse_datetime(current_datetime)
            current_hour = dt.hour + dt.minute / 60.0

            query = f"Please optimize the tourist itinerary given that the current time is {current_hour}. Analyze the conditions first, then provide the optimized schedule."

            result = self.agent_executor.invoke({"input": query})
            return result["output"]

        except Exception as e:
            return f"Error optimizing schedule: {str(e)}"


In [32]:
schedule_data = [
    {
        "name": "Cairo Tower",
        "lat": 30.0459751,
        "lon": 31.2242988,
        "outdoor": False,
        "hours_needed": 3,
        "start_time": 10,
        "end_time": 13,
        "day_text": "Monday",
        "venue_open": 9,
        "venue_closed": 1,  # 1 AM next day
        "day_raw": [0,0,0,20,25,30,35,35,40,45,55,65,75,80,80,70,60,45,25,0,0,0,0,0],
        "date": "2025-08-04",
        "temperatures": [26.3,25.7,25.1,24.5,23.9,23.5,23.2,23.3,24.3,26.1,27.7,29.4,30.9,32.1,33.0,33.6,34.2,34.0,33.8,33.2,32.0,30.2,28.5,27.4]
    },
    {
        "name": "The Egyptian Museum in Cairo",
        "lat": 30.0483167,
        "lon": 31.2336674,
        "outdoor": False,
        "hours_needed": 2.5,
        "start_time": 13.5,
        "end_time": 16,
        "day_text": "Monday",
        "venue_open": 9,
        "venue_closed": 17,
        "day_raw": [0,0,0,50,65,70,70,70,75,70,60,0,0,0,0,0,0,0,0,0,0,0,0,0],
        "date": "2025-08-04",
        "temperatures": [26.3,25.7,25.1,24.5,23.9,23.5,23.2,23.3,24.3,26.1,27.7,29.4,30.9,32.1,33.0,33.6,34.2,34.0,33.8,33.2,32.0,30.2,28.5,27.4]
    },
    {
        "name": "Grand Egyptian Museum (GEM)",
        "lat": 29.9943546,
        "lon": 31.1192991,
        "outdoor": False,
        "hours_needed": 1,
        "start_time": 16.5,
        "end_time": 17.5,
        "day_text": "Monday",
        "venue_open": 8,
        "venue_closed": 19,
        "day_raw": [0,0,10,15,25,40,50,60,65,65,55,40,30,0,0,0,0,0,0,0,0,0,0,0],
        "date": "2025-08-04",
        "temperatures": [26.1,25.4,24.8,24.2,23.7,23.3,23.1,23.0,23.9,25.5,27.2,28.8,30.3,31.7,32.6,33.4,33.8,33.8,33.5,32.9,31.7,29.7,28.2,27.1]
    }
]

In [46]:
api_key = "sk-your_deepseek_api_key"
agent = TouristItineraryAgent(api_key)
agent.setup_agent(schedule_data)

# Setup agent with schedule data
agent.setup_agent(schedule_data)

# Optimize schedule for current time
current_time = "2025-08-04 12:00"  # Example current time
result = agent.optimize_schedule(current_time)



> Entering new AgentExecutor chain...
